# THE BATTLE OF NEIGHBOURHOOD

# Introduction :

New York City's demographics show that it is a large and ethnically diverse metropolis. With it's diverse culture , comes diverse food items. There are many resturants in New york City, each beloning to different categories like Chinese , Indian , French etc.

So as part of this project , we will list and visualize all major parts of New York City that has great indian resturants.

# Data :

For this project we need the following data:

 1.New York City data that contains list Boroughs, Neighborhoods along with their latitude and longitude.  
 2.Indian resturants in each neighborhood of new york city.  
 3.GeoSpace data  

# Questions that will be answers at the end of this project :

# 1. What are best location in New York City for Indian Cuisine?
# 2. Which areas have potential Indian Restaurant Market?
# 3. Which all areas lack Indian Restaurants?
# 4. Which is the best place to stay if you prefer Indian Cuisine?

# Methodology:

1.We begin by collecting the New York city data from "https://cocl.us/new_york_dataset".  
2.We will find all venues for each neighborhood using FourSquare API.  
3.We will then filter out all Indian Restuarant venues.  
4.Next using FourSquare API, we will find the Ratings, Tips, and Like count for all the Indian Resturants.  
5.Next we will sort the data keeping Ratings as the constraint.  
6.Finally, we will visualize the Ranking of neighborhoods using python's Folium library.  

Lets Start by importing the required Libraries.

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from bs4 import BeautifulSoup
#!pip install geocoder
import geocoder
import os
#!pip install folium
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline


print('Libraries imported.')


Libraries imported.  

Now we define a function to get the geocodes i.e latitude and longitude of a given location using geopy.

In [17]:
def geo_location(address):
    # get geo location of address
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    return latitude,longitude

We define a function to intract with FourSquare API and get top 100 venues within a radius of 1000 metres for a given latitude and 
longitude. Below function will return us the venue id , venue name and category.

In [18]:
def get_venues(lat,lng):
    
    #set variables
    radius=1000
    LIMIT=100
    CLIENT_ID = ['CLIENT_ID'] # your Foursquare ID
    CLIENT_SECRET = ['CLIENT_SECRET'] # your Foursquare Secret
    VERSION = '20200401' # Foursquare API version
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results["response"]['groups'][0]['items']
    venue_details=[]
    for row in venue_data:
        try:
            venue_id=row['venue']['id']
            venue_name=row['venue']['name']
            venue_category=row['venue']['categories'][0]['name']
            venue_details.append([venue_id,venue_name,venue_category])
        except KeyError:
            pass
        
    column_names=['ID','Name','Category']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

Now we will define a function to get venue details like like count , rating , tip counts for a given venue id. This will be used for ranking.

In [19]:
def get_venue_details(venue_id):
        
    CLIENT_ID = ['CLIENT_ID'] # your Foursquare ID
    CLIENT_SECRET = ['CLIENT_SECRET'] # your Foursquare Secret
    VERSION = '20200401' # Foursquare API version
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/{}?&client_id={}&client_secret={}&v={}'.format(
            venue_id,
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
    
    # get all the data
    results = requests.get(url).json()
    venue_data=results['response']['venue']
    venue_details=[]
    try:
        venue_id=venue_data['id']
        venue_name=venue_data['name']
        venue_likes=venue_data['likes']['count']
        venue_rating=venue_data['rating']
        venue_tips=venue_data['tips']['count']
        venue_details.append([venue_id,venue_name,venue_likes,venue_rating,venue_tips])
    except KeyError:
        pass
        
    column_names=['ID','Name','Likes','Rating','Tips']
    df = pd.DataFrame(venue_details,columns=column_names)
    return df

Now we define a funtion to get the new york city data such as Boroughs, Neighborhoods along with their latitude and longitude.

In [20]:
def get_new_york_data():
    url='https://cocl.us/new_york_dataset'
    resp=requests.get(url).json()
    # all data is present in features label
    features=resp['features']
    
    # define the dataframe columns
    column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
    # instantiate the dataframe
    new_york_data = pd.DataFrame(columns=column_names)
    
    for data in features:
        borough = data['properties']['borough'] 
        neighborhood_name = data['properties']['name']
        
        neighborhood_latlon = data['geometry']['coordinates']
        neighborhood_lat = neighborhood_latlon[1]
        neighborhood_lon = neighborhood_latlon[0]
    
        new_york_data = new_york_data.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    return new_york_data

We will call the above funtion to get the new york city data.

In [21]:
new_york_data=get_new_york_data()

In [22]:
new_york_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [23]:
new_york_data.shape

(306, 4)

# The above result shows that there are 306 different Neighborhoods in New York.

Now let create a BAR PLOT to show different Neighborhoods in New York.

In [ ]:
plt.figure(figsize=(9,5), dpi = 100)
# title
plt.title('Number of Neighborhood for each Borough in New York City')
#On x-axis
plt.xlabel('Borough', fontsize = 15)
#On y-axis
plt.ylabel('No.of Neighborhood', fontsize=15)
#giving a bar plot
new_york_data.groupby('Borough')['Neighborhood'].count().plot(kind='bar')
#legend
plt.legend()
#displays the plot
plt.show()

# From the above Bar Plot, we can see that Queens has highest number of neighborhoods.

Next we will collect Indian resturants for each Neighborhood.

In [ ]:
# prepare neighborhood list that contains indian resturants
column_names=['Borough', 'Neighborhood', 'ID','Name']
indian_rest_ny=pd.DataFrame(columns=column_names)
count=1
for row in new_york_data.values.tolist():
    Borough, Neighborhood, Latitude, Longitude=row
    venues = get_venues(Latitude,Longitude)
    indian_resturants=venues[venues['Category']=='Indian Restaurant']   
    print('(',count,'/',len(new_york_data),')','Indian Resturants in '+Neighborhood+', '+Borough+':'+str(len(indian_resturants)))
    for resturant_detail in indian_resturants.values.tolist():
        id, name , category=resturant_detail
        indian_rest_ny = indian_rest_ny.append({'Borough': Borough,
                                                'Neighborhood': Neighborhood, 
                                                'ID': id,
                                                'Name' : name
                                               }, ignore_index=True)
    count+=1

( 1 / 306 ) Indian Resturants in Wakefield, Bronx:0  
( 2 / 306 ) Indian Resturants in Co-op City, Bronx:0  
( 3 / 306 ) Indian Resturants in Eastchester, Bronx:0  
( 4 / 306 ) Indian Resturants in Fieldston, Bronx:0  
( 5 / 306 ) Indian Resturants in Riverdale, Bronx:0  
( 6 / 306 ) Indian Resturants in Kingsbridge, Bronx:0  
( 7 / 306 ) Indian Resturants in Marble Hill, Manhattan:0  
( 8 / 306 ) Indian Resturants in Woodlawn, Bronx:1  
( 9 / 306 ) Indian Resturants in Norwood, Bronx:0  
( 10 / 306 ) Indian Resturants in Williamsbridge, Bronx:0  
( 11 / 306 ) Indian Resturants in Baychester, Bronx:0  
( 12 / 306 ) Indian Resturants in Pelham Parkway, Bronx:0  
( 13 / 306 ) Indian Resturants in City Island, Bronx:0  
( 14 / 306 ) Indian Resturants in Bedford Park, Bronx:0  
( 15 / 306 ) Indian Resturants in University Heights, Bronx:0  
( 16 / 306 ) Indian Resturants in Morris Heights, Bronx:0  
( 17 / 306 ) Indian Resturants in Fordham, Bronx:0  
( 18 / 306 ) Indian Resturants in East Tremont, Bronx:0  
( 19 / 306 ) Indian Resturants in West Farms, Bronx:0  
( 20 / 306 ) Indian Resturants in High  Bridge, Bronx:0  
( 21 / 306 ) Indian Resturants in Melrose, Bronx:0  
( 22 / 306 ) Indian Resturants in Mott Haven, Bronx:0  
( 23 / 306 ) Indian Resturants in Port Morris, Bronx:0  
( 24 / 306 ) Indian Resturants in Longwood, Bronx:0  
( 25 / 306 ) Indian Resturants in Hunts Point, Bronx:0  
( 26 / 306 ) Indian Resturants in Morrisania, Bronx:0  
( 27 / 306 ) Indian Resturants in Soundview, Bronx:0  
( 28 / 306 ) Indian Resturants in Clason Point, Bronx:0  
( 29 / 306 ) Indian Resturants in Throgs Neck, Bronx:0  
( 30 / 306 ) Indian Resturants in Country Club, Bronx:0  
( 31 / 306 ) Indian Resturants in Parkchester, Bronx:1  
( 32 / 306 ) Indian Resturants in Westchester Square, Bronx:0  
( 33 / 306 ) Indian Resturants in Van Nest, Bronx:0  
( 34 / 306 ) Indian Resturants in Morris Park, Bronx:0  
( 35 / 306 ) Indian Resturants in Belmont, Bronx:0  
( 36 / 306 ) Indian Resturants in Spuyten Duyvil, Bronx:1  
( 37 / 306 ) Indian Resturants in North Riverdale, Bronx:0  
( 38 / 306 ) Indian Resturants in Pelham Bay, Bronx:0  
( 39 / 306 ) Indian Resturants in Schuylerville, Bronx:0  
( 40 / 306 ) Indian Resturants in Edgewater Park, Bronx:0  
( 41 / 306 ) Indian Resturants in Castle Hill, Bronx:0  
( 42 / 306 ) Indian Resturants in Olinville, Bronx:0  
( 43 / 306 ) Indian Resturants in Pelham Gardens, Bronx:0  
( 44 / 306 ) Indian Resturants in Concourse, Bronx:1  
( 45 / 306 ) Indian Resturants in Unionport, Bronx:2  
( 46 / 306 ) Indian Resturants in Edenwald, Bronx:0  
( 47 / 306 ) Indian Resturants in Bay Ridge, Brooklyn:1  
( 48 / 306 ) Indian Resturants in Bensonhurst, Brooklyn:0  
( 49 / 306 ) Indian Resturants in Sunset Park, Brooklyn:0  
( 50 / 306 ) Indian Resturants in Greenpoint, Brooklyn:0  
( 51 / 306 ) Indian Resturants in Gravesend, Brooklyn:0  
( 52 / 306 ) Indian Resturants in Brighton Beach, Brooklyn:1  
( 53 / 306 ) Indian Resturants in Sheepshead Bay, Brooklyn:0  
( 54 / 306 ) Indian Resturants in Manhattan Terrace, Brooklyn:0  
( 55 / 306 ) Indian Resturants in Flatbush, Brooklyn:2  
( 56 / 306 ) Indian Resturants in Crown Heights, Brooklyn:0  
( 57 / 306 ) Indian Resturants in East Flatbush, Brooklyn:1  
( 58 / 306 ) Indian Resturants in Kensington, Brooklyn:2  
( 59 / 306 ) Indian Resturants in Windsor Terrace, Brooklyn:0  
( 60 / 306 ) Indian Resturants in Prospect Heights, Brooklyn:0  
( 61 / 306 ) Indian Resturants in Brownsville, Brooklyn:0  
( 62 / 306 ) Indian Resturants in Williamsburg, Brooklyn:0  
( 63 / 306 ) Indian Resturants in Bushwick, Brooklyn:0  
( 64 / 306 ) Indian Resturants in Bedford Stuyvesant, Brooklyn:0  
( 65 / 306 ) Indian Resturants in Brooklyn Heights, Brooklyn:0  
( 66 / 306 ) Indian Resturants in Cobble Hill, Brooklyn:0  
( 67 / 306 ) Indian Resturants in Carroll Gardens, Brooklyn:0  
( 68 / 306 ) Indian Resturants in Red Hook, Brooklyn:0  
( 69 / 306 ) Indian Resturants in Gowanus, Brooklyn:1  
( 70 / 306 ) Indian Resturants in Fort Greene, Brooklyn:1  
( 71 / 306 ) Indian Resturants in Park Slope, Brooklyn:0  
( 72 / 306 ) Indian Resturants in Cypress Hills, Brooklyn:0  
( 73 / 306 ) Indian Resturants in East New York, Brooklyn:0  
( 74 / 306 ) Indian Resturants in Starrett City, Brooklyn:0  
( 75 / 306 ) Indian Resturants in Canarsie, Brooklyn:0  
( 76 / 306 ) Indian Resturants in Flatlands, Brooklyn:0  
( 77 / 306 ) Indian Resturants in Mill Island, Brooklyn:0  
( 78 / 306 ) Indian Resturants in Manhattan Beach, Brooklyn:0  
( 79 / 306 ) Indian Resturants in Coney Island, Brooklyn:0  
( 80 / 306 ) Indian Resturants in Bath Beach, Brooklyn:0  
( 81 / 306 ) Indian Resturants in Borough Park, Brooklyn:0  
( 82 / 306 ) Indian Resturants in Dyker Heights, Brooklyn:0  
( 83 / 306 ) Indian Resturants in Gerritsen Beach, Brooklyn:0  
( 84 / 306 ) Indian Resturants in Marine Park, Brooklyn:0  
( 85 / 306 ) Indian Resturants in Clinton Hill, Brooklyn:2  
( 86 / 306 ) Indian Resturants in Sea Gate, Brooklyn:0  
( 87 / 306 ) Indian Resturants in Downtown, Brooklyn:0  
( 88 / 306 ) Indian Resturants in Boerum Hill, Brooklyn:0  
( 89 / 306 ) Indian Resturants in Prospect Lefferts Gardens, Brooklyn:2  
( 90 / 306 ) Indian Resturants in Ocean Hill, Brooklyn:2  
( 91 / 306 ) Indian Resturants in City Line, Brooklyn:0  
( 92 / 306 ) Indian Resturants in Bergen Beach, Brooklyn:0  
( 93 / 306 ) Indian Resturants in Midwood, Brooklyn:0  
( 94 / 306 ) Indian Resturants in Prospect Park South, Brooklyn:2  
( 95 / 306 ) Indian Resturants in Georgetown, Brooklyn:0  
( 96 / 306 ) Indian Resturants in East Williamsburg, Brooklyn:0  
( 97 / 306 ) Indian Resturants in North Side, Brooklyn:1  
( 98 / 306 ) Indian Resturants in South Side, Brooklyn:1  
( 99 / 306 ) Indian Resturants in Ocean Parkway, Brooklyn:0  
( 100 / 306 ) Indian Resturants in Fort Hamilton, Brooklyn:0  
( 101 / 306 ) Indian Resturants in Chinatown, Manhattan:0  
( 102 / 306 ) Indian Resturants in Washington Heights, Manhattan:1  
( 103 / 306 ) Indian Resturants in Inwood, Manhattan:0  
( 104 / 306 ) Indian Resturants in Hamilton Heights, Manhattan:2  
( 105 / 306 ) Indian Resturants in Manhattanville, Manhattan:2  
( 106 / 306 ) Indian Resturants in Central Harlem, Manhattan:2  
( 107 / 306 ) Indian Resturants in East Harlem, Manhattan:1  
( 108 / 306 ) Indian Resturants in Upper East Side, Manhattan:0  
( 109 / 306 ) Indian Resturants in Yorkville, Manhattan:1  
( 110 / 306 ) Indian Resturants in Lenox Hill, Manhattan:0  
( 111 / 306 ) Indian Resturants in Roosevelt Island, Manhattan:1  
( 112 / 306 ) Indian Resturants in Upper West Side, Manhattan:2  
( 113 / 306 ) Indian Resturants in Lincoln Square, Manhattan:0  
( 114 / 306 ) Indian Resturants in Clinton, Manhattan:0  
( 115 / 306 ) Indian Resturants in Midtown, Manhattan:1  
( 116 / 306 ) Indian Resturants in Murray Hill, Manhattan:1  
( 117 / 306 ) Indian Resturants in Chelsea, Manhattan:1  
( 118 / 306 ) Indian Resturants in Greenwich Village, Manhattan:0  
( 119 / 306 ) Indian Resturants in East Village, Manhattan:0  
( 120 / 306 ) Indian Resturants in Lower East Side, Manhattan:0  
( 121 / 306 ) Indian Resturants in Tribeca, Manhattan:1  
( 122 / 306 ) Indian Resturants in Little Italy, Manhattan:0  
( 123 / 306 ) Indian Resturants in Soho, Manhattan:0  
( 124 / 306 ) Indian Resturants in West Village, Manhattan:2  
( 125 / 306 ) Indian Resturants in Manhattan Valley, Manhattan:5  
( 126 / 306 ) Indian Resturants in Morningside Heights, Manhattan:1  
( 127 / 306 ) Indian Resturants in Gramercy, Manhattan:3  
( 128 / 306 ) Indian Resturants in Battery Park City, Manhattan:0  
( 129 / 306 ) Indian Resturants in Financial District, Manhattan:0  
( 130 / 306 ) Indian Resturants in Astoria, Queens:1  
( 131 / 306 ) Indian Resturants in Woodside, Queens:8  
( 132 / 306 ) Indian Resturants in Jackson Heights, Queens:6  
( 133 / 306 ) Indian Resturants in Elmhurst, Queens:2  
( 134 / 306 ) Indian Resturants in Howard Beach, Queens:0  
( 135 / 306 ) Indian Resturants in Corona, Queens:0  
( 136 / 306 ) Indian Resturants in Forest Hills, Queens:0  
( 137 / 306 ) Indian Resturants in Kew Gardens, Queens:2  
( 138 / 306 ) Indian Resturants in Richmond Hill, Queens:6  
( 139 / 306 ) Indian Resturants in Flushing, Queens:0  
( 140 / 306 ) Indian Resturants in Long Island City, Queens:2  
( 141 / 306 ) Indian Resturants in Sunnyside, Queens:1  
( 142 / 306 ) Indian Resturants in East Elmhurst, Queens:0  
( 143 / 306 ) Indian Resturants in Maspeth, Queens:0  
( 144 / 306 ) Indian Resturants in Ridgewood, Queens:1  
( 145 / 306 ) Indian Resturants in Glendale, Queens:0  
( 146 / 306 ) Indian Resturants in Rego Park, Queens:1  
( 147 / 306 ) Indian Resturants in Woodhaven, Queens:0  
( 148 / 306 ) Indian Resturants in Ozone Park, Queens:1  
( 149 / 306 ) Indian Resturants in South Ozone Park, Queens:0  
( 150 / 306 ) Indian Resturants in College Point, Queens:0  
( 151 / 306 ) Indian Resturants in Whitestone, Queens:0  
( 152 / 306 ) Indian Resturants in Bayside, Queens:3  
( 153 / 306 ) Indian Resturants in Auburndale, Queens:0  
( 154 / 306 ) Indian Resturants in Little Neck, Queens:0  
( 155 / 306 ) Indian Resturants in Douglaston, Queens:0  
( 156 / 306 ) Indian Resturants in Glen Oaks, Queens:4  
( 157 / 306 ) Indian Resturants in Bellerose, Queens:2  
( 158 / 306 ) Indian Resturants in Kew Gardens Hills, Queens:1  
( 159 / 306 ) Indian Resturants in Fresh Meadows, Queens:0  
( 160 / 306 ) Indian Resturants in Briarwood, Queens:3  
( 161 / 306 ) Indian Resturants in Jamaica Center, Queens:3  
( 162 / 306 ) Indian Resturants in Oakland Gardens, Queens:0  
( 163 / 306 ) Indian Resturants in Queens Village, Queens:0  
( 164 / 306 ) Indian Resturants in Hollis, Queens:0  
( 165 / 306 ) Indian Resturants in South Jamaica, Queens:0  
( 166 / 306 ) Indian Resturants in St. Albans, Queens:0  
( 167 / 306 ) Indian Resturants in Rochdale, Queens:0  
( 168 / 306 ) Indian Resturants in Springfield Gardens, Queens:0  
( 169 / 306 ) Indian Resturants in Cambria Heights, Queens:0  
( 170 / 306 ) Indian Resturants in Rosedale, Queens:0  
( 171 / 306 ) Indian Resturants in Far Rockaway, Queens:0  
( 172 / 306 ) Indian Resturants in Broad Channel, Queens:0  
( 173 / 306 ) Indian Resturants in Breezy Point, Queens:0  
( 174 / 306 ) Indian Resturants in Steinway, Queens:1  
( 175 / 306 ) Indian Resturants in Beechhurst, Queens:0  
( 176 / 306 ) Indian Resturants in Bay Terrace, Queens:0  
( 177 / 306 ) Indian Resturants in Edgemere, Queens:0  
( 178 / 306 ) Indian Resturants in Arverne, Queens:0  
( 179 / 306 ) Indian Resturants in Rockaway Beach, Queens:0  
( 180 / 306 ) Indian Resturants in Neponsit, Queens:0  
( 181 / 306 ) Indian Resturants in Murray Hill, Queens:0  
( 182 / 306 ) Indian Resturants in Floral Park, Queens:11  
( 183 / 306 ) Indian Resturants in Holliswood, Queens:1  
( 184 / 306 ) Indian Resturants in Jamaica Estates, Queens:2  
( 185 / 306 ) Indian Resturants in Queensboro Hill, Queens:0  
( 186 / 306 ) Indian Resturants in Hillcrest, Queens:0  
( 187 / 306 ) Indian Resturants in Ravenswood, Queens:1  
( 188 / 306 ) Indian Resturants in Lindenwood, Queens:0  
( 189 / 306 ) Indian Resturants in Laurelton, Queens:0  
( 190 / 306 ) Indian Resturants in Lefrak City, Queens:0  
( 191 / 306 ) Indian Resturants in Belle Harbor, Queens:0  
( 192 / 306 ) Indian Resturants in Rockaway Park, Queens:0  
( 193 / 306 ) Indian Resturants in Somerville, Queens:0  
( 194 / 306 ) Indian Resturants in Brookville, Queens:0  
( 195 / 306 ) Indian Resturants in Bellaire, Queens:1  
( 196 / 306 ) Indian Resturants in North Corona, Queens:0  
( 197 / 306 ) Indian Resturants in Forest Hills Gardens, Queens:0  
( 198 / 306 ) Indian Resturants in St. George, Staten Island:0  
( 199 / 306 ) Indian Resturants in New Brighton, Staten Island:1  
( 200 / 306 ) Indian Resturants in Stapleton, Staten Island:0  
( 201 / 306 ) Indian Resturants in Rosebank, Staten Island:0  
( 202 / 306 ) Indian Resturants in West Brighton, Staten Island:0  
( 203 / 306 ) Indian Resturants in Grymes Hill, Staten Island:0  
( 204 / 306 ) Indian Resturants in Todt Hill, Staten Island:0  
( 205 / 306 ) Indian Resturants in South Beach, Staten Island:0  
( 206 / 306 ) Indian Resturants in Port Richmond, Staten Island:0  
( 207 / 306 ) Indian Resturants in Mariner's Harbor, Staten Island:0  
( 208 / 306 ) Indian Resturants in Port Ivory, Staten Island:0  
( 209 / 306 ) Indian Resturants in Castleton Corners, Staten Island:0  
( 210 / 306 ) Indian Resturants in New Springville, Staten Island:0  
( 211 / 306 ) Indian Resturants in Travis, Staten Island:0  
( 212 / 306 ) Indian Resturants in New Dorp, Staten Island:1  
( 213 / 306 ) Indian Resturants in Oakwood, Staten Island:0  
( 214 / 306 ) Indian Resturants in Great Kills, Staten Island:0  
( 215 / 306 ) Indian Resturants in Eltingville, Staten Island:0  
( 216 / 306 ) Indian Resturants in Annadale, Staten Island:0  
( 217 / 306 ) Indian Resturants in Woodrow, Staten Island:0  
( 218 / 306 ) Indian Resturants in Tottenville, Staten Island:0  
( 219 / 306 ) Indian Resturants in Tompkinsville, Staten Island:1  
( 220 / 306 ) Indian Resturants in Silver Lake, Staten Island:0  
( 221 / 306 ) Indian Resturants in Sunnyside, Staten Island:1  
( 222 / 306 ) Indian Resturants in Ditmas Park, Brooklyn:3  
( 223 / 306 ) Indian Resturants in Wingate, Brooklyn:0  
( 224 / 306 ) Indian Resturants in Rugby, Brooklyn:0  
( 225 / 306 ) Indian Resturants in Park Hill, Staten Island:1  
( 226 / 306 ) Indian Resturants in Westerleigh, Staten Island:0  
( 227 / 306 ) Indian Resturants in Graniteville, Staten Island:0  
( 228 / 306 ) Indian Resturants in Arlington, Staten Island:0  
( 229 / 306 ) Indian Resturants in Arrochar, Staten Island:0  
( 230 / 306 ) Indian Resturants in Grasmere, Staten Island:0  
( 231 / 306 ) Indian Resturants in Old Town, Staten Island:0  
( 232 / 306 ) Indian Resturants in Dongan Hills, Staten Island:0  
( 233 / 306 ) Indian Resturants in Midland Beach, Staten Island:0  
( 234 / 306 ) Indian Resturants in Grant City, Staten Island:1  
( 235 / 306 ) Indian Resturants in New Dorp Beach, Staten Island:0  
( 236 / 306 ) Indian Resturants in Bay Terrace, Staten Island:0  
( 237 / 306 ) Indian Resturants in Huguenot, Staten Island:0  
( 238 / 306 ) Indian Resturants in Pleasant Plains, Staten Island:0  
( 239 / 306 ) Indian Resturants in Butler Manor, Staten Island:0  
( 240 / 306 ) Indian Resturants in Charleston, Staten Island:0  
( 241 / 306 ) Indian Resturants in Rossville, Staten Island:0  
( 242 / 306 ) Indian Resturants in Arden Heights, Staten Island:0  
( 243 / 306 ) Indian Resturants in Greenridge, Staten Island:0  
( 244 / 306 ) Indian Resturants in Heartland Village, Staten Island:0  
( 245 / 306 ) Indian Resturants in Chelsea, Staten Island:0  
( 246 / 306 ) Indian Resturants in Bloomfield, Staten Island:0  
( 247 / 306 ) Indian Resturants in Bulls Head, Staten Island:0  
( 248 / 306 ) Indian Resturants in Carnegie Hill, Manhattan:3  
( 249 / 306 ) Indian Resturants in Noho, Manhattan:0  
( 250 / 306 ) Indian Resturants in Civic Center, Manhattan:1  
( 251 / 306 ) Indian Resturants in Midtown South, Manhattan:1  
( 252 / 306 ) Indian Resturants in Richmond Town, Staten Island:0  
( 253 / 306 ) Indian Resturants in Shore Acres, Staten Island:0  
( 254 / 306 ) Indian Resturants in Clifton, Staten Island:0  
( 255 / 306 ) Indian Resturants in Concord, Staten Island:1  
( 256 / 306 ) Indian Resturants in Emerson Hill, Staten Island:1  
( 257 / 306 ) Indian Resturants in Randall Manor, Staten Island:0  
( 258 / 306 ) Indian Resturants in Howland Hook, Staten Island:0  
( 259 / 306 ) Indian Resturants in Elm Park, Staten Island:0  
( 260 / 306 ) Indian Resturants in Remsen Village, Brooklyn:0  
( 261 / 306 ) Indian Resturants in New Lots, Brooklyn:0  
( 262 / 306 ) Indian Resturants in Paerdegat Basin, Brooklyn:0  
( 263 / 306 ) Indian Resturants in Mill Basin, Brooklyn:0  
( 264 / 306 ) Indian Resturants in Jamaica Hills, Queens:3  
( 265 / 306 ) Indian Resturants in Utopia, Queens:0  
( 266 / 306 ) Indian Resturants in Pomonok, Queens:0  
( 267 / 306 ) Indian Resturants in Astoria Heights, Queens:0  
( 268 / 306 ) Indian Resturants in Claremont Village, Bronx:0  
( 269 / 306 ) Indian Resturants in Concourse Village, Bronx:1  
( 270 / 306 ) Indian Resturants in Mount Eden, Bronx:0  
( 271 / 306 ) Indian Resturants in Mount Hope, Bronx:0  
( 272 / 306 ) Indian Resturants in Sutton Place, Manhattan:3  
( 273 / 306 ) Indian Resturants in Hunters Point, Queens:0  
( 274 / 306 ) Indian Resturants in Turtle Bay, Manhattan:3  
( 275 / 306 ) Indian Resturants in Tudor City, Manhattan:2  
( 276 / 306 ) Indian Resturants in Stuyvesant Town, Manhattan:0  
( 277 / 306 ) Indian Resturants in Flatiron, Manhattan:0  
( 278 / 306 ) Indian Resturants in Sunnyside Gardens, Queens:1  
( 279 / 306 ) Indian Resturants in Blissville, Queens:1  
( 280 / 306 ) Indian Resturants in Fulton Ferry, Brooklyn:0  
( 281 / 306 ) Indian Resturants in Vinegar Hill, Brooklyn:0  
( 282 / 306 ) Indian Resturants in Weeksville, Brooklyn:0  
( 283 / 306 ) Indian Resturants in Broadway Junction, Brooklyn:1  
( 284 / 306 ) Indian Resturants in Dumbo, Brooklyn:0  
( 285 / 306 ) Indian Resturants in Manor Heights, Staten Island:0  
( 286 / 306 ) Indian Resturants in Willowbrook, Staten Island:0  
( 287 / 306 ) Indian Resturants in Sandy Ground, Staten Island:0  
( 288 / 306 ) Indian Resturants in Egbertville, Staten Island:0  
( 289 / 306 ) Indian Resturants in Roxbury, Queens:0  
( 290 / 306 ) Indian Resturants in Homecrest, Brooklyn:0  
( 291 / 306 ) Indian Resturants in Middle Village, Queens:0  
( 292 / 306 ) Indian Resturants in Prince's Bay, Staten Island:0  
( 293 / 306 ) Indian Resturants in Lighthouse Hill, Staten Island:0  
( 294 / 306 ) Indian Resturants in Richmond Valley, Staten Island:0  
( 295 / 306 ) Indian Resturants in Malba, Queens:0  
( 296 / 306 ) Indian Resturants in Highland Park, Brooklyn:0  
( 297 / 306 ) Indian Resturants in Madison, Brooklyn:0  
( 298 / 306 ) Indian Resturants in Bronxdale, Bronx:0  
( 299 / 306 ) Indian Resturants in Allerton, Bronx:0  
( 300 / 306 ) Indian Resturants in Kingsbridge Heights, Bronx:0  
( 301 / 306 ) Indian Resturants in Erasmus, Brooklyn:1  
( 302 / 306 ) Indian Resturants in Hudson Yards, Manhattan:0  
( 303 / 306 ) Indian Resturants in Hammels, Queens:0  
( 304 / 306 ) Indian Resturants in Bayswater, Queens:0  
( 305 / 306 ) Indian Resturants in Queensbridge, Queens:2  
( 306 / 306 ) Indian Resturants in Fox Hills, Staten Island:1    

Now that we have got all the indian resturants in new york city , we will analyze it.

In [27]:
indian_rest_ny.head()

,Borough,Neighborhood,ID,Name


In [28]:
indian_rest_ny.shape

(0, 4)

# From the above result, we see that there are 151 Indian Resturants across New York City.

Now let create a BAR PLOT to show Number of Indian Resturants for each Borough in New York City.

In [ ]:
plt.figure(figsize=(9,5), dpi = 100)
# title
plt.title('Number of Indian Resturants for each Borough in New York City')
#On x-axis
plt.xlabel('Borough', fontsize = 15)
#On y-axis
plt.ylabel('No.of Indian Resturants', fontsize=15)
#giving a bar plot
indian_rest_ny.groupby('Borough')['ID'].count().plot(kind='bar')
#legend
plt.legend()
#displays the plot
plt.show()

In [30]:
indian_rest_ny[indian_rest_ny['Neighborhood']=='Floral Park']

,Borough,Neighborhood,ID,Name


 ID        Name  Likes  Rating  Tips
0  4c0448d9310fc9b6bf1dc761  Curry Spot      5     7.6    10
( 1 / 151 ) processed
                         ID                         Name  Likes  Rating  Tips
0  4c194631838020a13e78e561  Melanies Roti Bar And Grill      3     5.8     2
( 2 / 151 ) processed
                         ID                  Name  Likes  Rating  Tips
0  4c04544df423a593ac83d116  Cumin Indian Cuisine     13     6.1     9
( 3 / 151 ) processed
                         ID         Name  Likes  Rating  Tips
0  551b7f75498e86c00a0ed2e1  Hungry Bird      8     6.9     3
( 4 / 151 ) processed
                         ID                         Name  Likes  Rating  Tips
0  4c194631838020a13e78e561  Melanies Roti Bar And Grill      3     5.8     2
( 5 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 55dfa36a498e164ef19bef7b
( 6 / 151 ) processed
                         ID       Name  Likes  Rating  Tips
0  4b5a4dc8f964a520a2bb28e3  Taj Mahal     38     8.4    26
( 7 / 151 ) processed
                         ID          Name  Likes  Rating  Tips
0  4af0d31bf964a5207ddf21e3  Pak Nasheman      9     7.5     4
( 8 / 151 ) processed
                         ID          Name  Likes  Rating  Tips
0  52213c4211d295d4c57a607c  Ashoka Grill      8     7.0    14
( 9 / 151 ) processed
                         ID                Name  Likes  Rating  Tips
0  564d283d498e6e851df79d87  Great Indian Curry      3     6.8     2
( 10 / 151 ) processed
                         ID          Name  Likes  Rating  Tips
0  512a9ea9e4b004fb8eeb84e5  Silver Krust     12     8.1     3
( 11 / 151 ) processed
                         ID       Name  Likes  Rating  Tips
0  4db0f4371e729fcc56497f20  Mashallah     19     7.5     7
( 12 / 151 ) processed
                         ID                          Name  Likes  Rating  Tips
0  4b718914f964a520c04b2de3  Madina Restaurant and Sweets     17     6.5    12
( 13 / 151 ) processed
                         ID                       Name  Likes  Rating  Tips
0  52f18573498ec2c34e830ffd  Kanan's Indian Restaurant     23     7.7     8
( 14 / 151 ) processed
                         ID         Name  Likes  Rating  Tips
0  57596dad498e732300496b23  Dosa Royale     75     8.7    22
( 15 / 151 ) processed
                         ID         Name  Likes  Rating  Tips
0  57596dad498e732300496b23  Dosa Royale     75     8.7    22
( 16 / 151 ) processed
                         ID           Name  Likes  Rating  Tips
0  568d3902498e619efcbc3f58  Spice & Grill     19     7.6     6
( 17 / 151 ) processed
                         ID                        Name  Likes  Rating  Tips
0  4bb93b70cf2fc9b6fe64a002  Gandhi Fine Indian Cuisine     81     8.5    47
( 18 / 151 ) processed
                         ID             Name  Likes  Rating  Tips
0  4afdf78bf964a520862c22e3  King of Tandoor     25     7.3    23
( 19 / 151 ) processed
                         ID             Name  Likes  Rating  Tips
0  5539753f498edbace4746b67  Tandoori Masala     12     7.7     2
( 20 / 151 ) processed
                         ID           Name  Likes  Rating  Tips
0  4f6cae2ee4b0d4a5afcef5c0  Delhi Heights     22     7.9     8
( 21 / 151 ) processed
                         ID                     Name  Likes  Rating  Tips
0  519ff6c8498e1300ddcbd45c  Anarkali Indian Cuisine     14     7.8     8
( 22 / 151 ) processed
                         ID       Name  Likes  Rating  Tips
0  4db0f4371e729fcc56497f20  Mashallah     19     7.5     7
( 23 / 151 ) processed
                         ID                Name  Likes  Rating  Tips
0  5631511b498e3d6d7e0a4df0  Tikka Indian Grill     95     8.5    29
( 24 / 151 ) processed
                         ID                Name  Likes  Rating  Tips
0  5631511b498e3d6d7e0a4df0  Tikka Indian Grill     95     8.5    29
( 25 / 151 ) processed
                         ID                      Name  Likes  Rating  Tips
0  4ae7876ef964a5201eac21e3  Kismat Indian Restaurant     45     7.7    25
( 26 / 151 ) processed
                         ID                           Name  Likes  Rating  \
0  54c2bd96498eaf5142e3fe92  Clove Indian Restaurant & Bar     32     7.4   

   Tips  
0    16  
( 27 / 151 ) processed
                         ID           Name  Likes  Rating  Tips
0  5914ff32b23dfa207eca38de  Mumbai Masala     14     7.3     7
( 28 / 151 ) processed
                         ID                 Name  Likes  Rating  Tips
0  529d382a11d2dd5ef107e641  Chapati House - NYC     73     7.8    18
( 29 / 151 ) processed
                         ID                           Name  Likes  Rating  \
0  54c2bd96498eaf5142e3fe92  Clove Indian Restaurant & Bar     32     7.4   

   Tips  
0    16  
( 30 / 151 ) processed
                         ID          Name  Likes  Rating  Tips
0  56d87f3d498ee215abee5724  Delhi Masala     13     8.7     3
( 31 / 151 ) processed
                         ID                           Name  Likes  Rating  \
0  54c2bd96498eaf5142e3fe92  Clove Indian Restaurant & Bar     32     7.4   

   Tips  
0    16  
( 32 / 151 ) processed
                         ID      Name  Likes  Rating  Tips
0  519fe6f5498e30595d370c44  Bawarchi      9     7.4     4
( 33 / 151 ) processed
                         ID            Name  Likes  Rating  Tips
0  5272ca4511d22488f6895caf  Drunken Munkey    208     8.6    61
( 34 / 151 ) processed
                         ID              Name  Likes  Rating  Tips
0  4fe4fb50c2eee335e4fea69d  Moti Mahal Delux    185     8.4    82
( 35 / 151 ) processed
                         ID    Name  Likes  Rating  Tips
0  42489a80f964a5208b201fe3  Swagat    105     8.4    38
( 36 / 151 ) processed
                         ID           Name  Likes  Rating  Tips
0  4b0dec08f964a520ae5223e3  Alachi Masala     18     7.7    17
( 37 / 151 ) processed
                         ID                   Name  Likes  Rating  Tips
0  49d91c12f964a520015e1fe3  The Kati Roll Company    837     8.8   260
( 38 / 151 ) processed
                         ID                   Name  Likes  Rating  Tips
0  59fcd48c464d6567ed2f5e37  The Kati Roll Company     18     8.6     2
( 39 / 151 ) processed
                         ID               Name  Likes  Rating  Tips
0  4b4aab62f964a520978c26e3  Dil-e Punjab Deli    106     8.7    43
( 40 / 151 ) processed
                         ID              Name  Likes  Rating  Tips
0  4bbb9dbded7776b0e1ad3e51  Tamarind TriBeCa    589     9.1   146
( 41 / 151 ) processed
                         ID                Name  Likes  Rating  Tips
0  5b5a2c9e66f3cd002ca0aab5  The Drunken Munkey     26     8.8     4
( 42 / 151 ) processed
                         ID  Name  Likes  Rating  Tips
0  591794df2be42556988e4a8e  Rahi    106     8.8    19
( 43 / 151 ) processed
                         ID        Name  Likes  Rating  Tips
0  4d24b812836f5481518645f7  Doaba Deli     41     7.5    13
( 44 / 151 ) processed
                         ID              Name  Likes  Rating  Tips
0  424de080f964a520ae201fe3  Manhattan Valley     42     7.9    30
( 45 / 151 ) processed
                         ID   Name  Likes  Rating  Tips
0  538ba1f2498e279098e4210a  Awadh     51     8.0    13
( 46 / 151 ) processed
                         ID       Name  Likes  Rating  Tips
0  49c5ad0af964a5201b571fe3  Roti Roll    114     7.9    58
( 47 / 151 ) processed
                         ID    Name  Likes  Rating  Tips
0  4ad12c9df964a52040dd20e3  Aangan     35     7.4    14
( 48 / 151 ) processed
                         ID       Name  Likes  Rating  Tips
0  49c5ad0af964a5201b571fe3  Roti Roll    114     7.9    58
( 49 / 151 ) processed
                         ID                 Name  Likes  Rating  Tips
0  4a70a75bf964a52016d81fe3  Bhatti Indian Grill    416     8.7   159
( 50 / 151 ) processed
                         ID   Name  Likes  Rating  Tips
0  57f7cbbe498edf8f07c7ba83  Sahib     59     8.8    13
( 51 / 151 ) processed
                         ID     Name  Likes  Rating  Tips
0  523b2b42498e1dfabcc8ab15  Pippali    303     8.7    77
( 52 / 151 ) processed
                         ID                 Name  Likes  Rating  Tips
0  4aa56c81f964a5204e4820e3  Seva Indian Cuisine    238     9.0   131
( 53 / 151 ) processed
                         ID            Name  Likes  Rating  Tips
0  4afdcf29f964a520162b22e3  Rajbhog Sweets     37     8.4    24
( 54 / 151 ) processed
                         ID      Name  Likes  Rating  Tips
0  4c7060e734443704ca0e245f  Kababish     30     7.7    17
( 55 / 151 ) processed
                         ID                      Name  Likes  Rating  Tips
0  4b998d3ff964a520fc8235e3  Maharaja Sweets & Snacks     52     8.1    24
( 56 / 151 ) processed
                         ID                     Name  Likes  Rating  Tips
0  5da4fbf091a08400078be19d  Angel Indian Restaurant      3     8.1     1
( 57 / 151 ) processed
                         ID           Name  Likes  Rating  Tips
0  3fd66200f964a52009f11ee3  Jackson Diner    141     7.9    77
( 58 / 151 ) processed
                         ID           Name  Likes  Rating  Tips
0  49ebb18ff964a52016671fe3  Delhi Heights     34     7.9    22
( 59 / 151 ) processed
                         ID  Name  Likes  Rating  Tips
0  4b189424f964a52043d423e3  Dera     47     7.4    19
( 60 / 151 ) processed
                         ID                         Name  Likes  Rating  Tips
0  4e334b7bb0fb17f64f81a8b2  Premium Sweets & Restaurant     11     7.4     5
( 61 / 151 ) processed
                         ID     Name  Likes  Rating  Tips
0  527d9cbc498edf0db10bde6b  Samudra     48     8.5    16
( 62 / 151 ) processed
                         ID            Name  Likes  Rating  Tips
0  4afdcf29f964a520162b22e3  Rajbhog Sweets     37     8.4    24
( 63 / 151 ) processed
                         ID                      Name  Likes  Rating  Tips
0  4b998d3ff964a520fc8235e3  Maharaja Sweets & Snacks     52     8.1    24
( 64 / 151 ) processed
                         ID                     Name  Likes  Rating  Tips
0  5da4fbf091a08400078be19d  Angel Indian Restaurant      3     8.1     1
( 65 / 151 ) processed
                         ID           Name  Likes  Rating  Tips
0  3fd66200f964a52009f11ee3  Jackson Diner    141     7.9    77
( 66 / 151 ) processed
                         ID           Name  Likes  Rating  Tips
0  49ebb18ff964a52016671fe3  Delhi Heights     34     7.9    22
( 67 / 151 ) processed
                         ID     Name  Likes  Rating  Tips
0  527d9cbc498edf0db10bde6b  Samudra     48     8.5    16
( 68 / 151 ) processed
                         ID                     Name  Likes  Rating  Tips
0  5da4fbf091a08400078be19d  Angel Indian Restaurant      3     8.1     1
( 69 / 151 ) processed
                         ID                Name  Likes  Rating  Tips
0  5782c9ce498edde587f5aa14  Tikka Indian Grill     10     7.8     2
( 70 / 151 ) processed
                         ID                   Name  Likes  Rating  Tips
0  4bbe78bfba9776b070cefdc8  Mehak Mughlai Cuisine      8     7.7    10
( 71 / 151 ) processed
                         ID          Name  Likes  Rating  Tips
0  4b522029f964a520f26927e3  Sohna Punjab      8     7.7     2
( 72 / 151 ) processed
                         ID                Name  Likes  Rating  Tips
0  5782c9ce498edde587f5aa14  Tikka Indian Grill     10     7.8     2
( 73 / 151 ) processed
                         ID        Name  Likes  Rating  Tips
0  4f1b77d7e4b044fd359e6d21  India Cafe      6     7.1     3
( 74 / 151 ) processed
                         ID                 Name  Likes  Rating  Tips
0  4adbaef0f964a520ff2921e3  Punjabi Kabab House      7     6.0     7
( 75 / 151 ) processed
                         ID                Name  Likes  Rating  Tips
0  4babc24ef964a5200ac73ae3  Royal India Palace      7     5.9     3
( 76 / 151 ) processed
                         ID          Name  Likes  Rating  Tips
0  4bb2bc48a32876b02e0b01fe  Tandoori Hut      6     5.9    15
( 77 / 151 ) processed
                         ID  Name  Likes  Rating  Tips
0  5b931ea69d7468002c3b1382  Adda    113     9.0    34
( 78 / 151 ) processed
                         ID                  Name  Likes  Rating  Tips
0  50a287a7e4b0033f830f06db  Raj's Indian Kitchen     22     7.3     9
( 79 / 151 ) processed
                         ID  Name  Likes  Rating  Tips
0  5b931ea69d7468002c3b1382  Adda    113     9.0    34
( 80 / 151 ) processed
                         ID                        Name  Likes  Rating  Tips
0  50e1c9708aca7ff2b3e50353  Nepalese Indian Restaurant     55     8.3    18
( 81 / 151 ) processed
                         ID               Name  Likes  Rating  Tips
0  5625af69498ebbc62b61a382  Aaheli Restaurant     12     7.8     6
( 82 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 572d7c21498e098658a0edd7
( 83 / 151 ) processed
                         ID                         Name  Likes  Rating  Tips
0  4f1f4996e4b01ff351a7a50c  Ayna Agra Indian Restaurant     39     8.2    12
( 84 / 151 ) processed
                         ID        Name  Likes  Rating  Tips
0  539a4ff0498e79c6745baba9  Masala Box      9     8.0     2
( 85 / 151 ) processed
                         ID       Name  Likes  Rating  Tips
0  539e27b0498e2eba582085ee  masalabox      7     6.5     2
( 86 / 151 ) processed
                         ID                       Name  Likes  Rating  Tips
0  4b787c49f964a5209cd12ee3  Santoor Indian Restaurant     37     7.6    18
( 87 / 151 ) processed
                         ID             Name  Likes  Rating  Tips
0  4b9030abf964a520397b33e3  Taste of Cochin      9     6.9     4
( 88 / 151 ) processed
                         ID           Name  Likes  Rating  Tips
0  4df0f39dd4c04d0392c853ea  Sagar Chinese      7     6.0     6
( 89 / 151 ) processed
                         ID          Name  Likes  Rating  Tips
0  4c0e256ab1b676b06589e186  Sohna Punjab      3     5.9     3
( 90 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4e6bfe1c7d8b2c711b17bbe5
( 91 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4dc0aaedae606fe8b71c226b
( 92 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4b96926df964a520abd534e3
( 93 / 151 ) processed
                         ID          Name  Likes  Rating  Tips
0  4bad49a0f964a52041423be3  Annam Brahma     15     8.0     3
( 94 / 151 ) processed
                         ID                               Name  Likes  Rating  \
0  4be334312fc7d13a7a81083a  Taj Mahal Restaurant & Party Hall     24     7.4   

   Tips  
0     5  
( 95 / 151 ) processed
                         ID            Name  Likes  Rating  Tips
0  4cc08b0900d83704ed474b5c  Sybil's Bakery      8     7.2     3
( 96 / 151 ) processed
                         ID              Name  Likes  Rating  Tips
0  4c434b2bd691c9b6ef8f8f0a  Sagar Restaurant     10     7.5     6
( 97 / 151 ) processed
                         ID          Name  Likes  Rating  Tips
0  4bad49a0f964a52041423be3  Annam Brahma     15     8.0     3
( 98 / 151 ) processed
                         ID            Name  Likes  Rating  Tips
0  4cc08b0900d83704ed474b5c  Sybil's Bakery      8     7.2     3
( 99 / 151 ) processed
                         ID                    Name  Likes  Rating  Tips
0  562035d9498e2abb4137c2c7  Yaar Indian Restaurant     31     8.6     7
( 100 / 151 ) processed
                         ID           Name  Likes  Rating  Tips
0  527ffc0811d2d329d5e49abd  Jackson Diner      9     8.0     3
( 101 / 151 ) processed
                         ID                      Name  Likes  Rating  Tips
0  4b647b56f964a520c4b62ae3  Usha Foods & Usha Sweets     33     7.9    10
( 102 / 151 ) processed
                         ID            Name  Likes  Rating  Tips
0  4e4e3e22bd4101d0d7a5c2d1  Kerala Kitchen      5     8.0     6
( 103 / 151 ) processed
                         ID                       Name  Likes  Rating  Tips
0  4b787c49f964a5209cd12ee3  Santoor Indian Restaurant     37     7.6    18
( 104 / 151 ) processed
                         ID           Name  Likes  Rating  Tips
0  4c0c01e0bbc676b00d6b4cd5  Mumbai Xpress     16     6.6    18
( 105 / 151 ) processed
                         ID             Name  Likes  Rating  Tips
0  4c76ff35a5676dcb72671721  Flavor Of India      6     6.1     6
( 106 / 151 ) processed
                         ID           Name  Likes  Rating  Tips
0  4df0f39dd4c04d0392c853ea  Sagar Chinese      7     6.0     6
( 107 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4c953a7672dd224bd8d1a191
( 108 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4cc642ed306e224b5bf2a76c
( 109 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 51d84192498ea979a3c4f13d
( 110 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4e6bfe1c7d8b2c711b17bbe5
( 111 / 151 ) processed
                         ID                        Name  Likes  Rating  Tips
0  4f580c1be4b0bdfd0e7e8102  Rajdhani Indian Restaurant     13     8.3     6
( 112 / 151 ) processed
                         ID              Name  Likes  Rating  Tips
0  4c434b2bd691c9b6ef8f8f0a  Sagar Restaurant     10     7.5     6
( 113 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4b8d5734f964a520b1f532e3
( 114 / 151 ) processed
                         ID                  Name  Likes  Rating  Tips
0  5623f6f9498e5a44a08bfae8  Boishakhi Restaurant     17     7.9     7
( 115 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4dc0aaedae606fe8b71c226b
( 116 / 151 ) processed
                         ID         Name  Likes  Rating  Tips
0  4b9be038f964a520393036e3  Dosa Garden     18     7.8    17
( 117 / 151 ) processed
                         ID               Name  Likes  Rating  Tips
0  4be74a502468c928505a0243  Taste Of India II     29     8.1    14
( 118 / 151 ) processed
                         ID         Name  Likes  Rating  Tips
0  4b9be038f964a520393036e3  Dosa Garden     18     7.8    17
( 119 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4b65f2e3f964a5206e0a2be3
( 120 / 151 ) processed
                         ID                     Name  Likes  Rating  Tips
0  519ff6c8498e1300ddcbd45c  Anarkali Indian Cuisine     14     7.8     8
( 121 / 151 ) processed
                         ID       Name  Likes  Rating  Tips
0  4db0f4371e729fcc56497f20  Mashallah     19     7.5     7
( 122 / 151 ) processed
                         ID                Name  Likes  Rating  Tips
0  564d283d498e6e851df79d87  Great Indian Curry      3     6.8     2
( 123 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4b65f2e3f964a5206e0a2be3
( 124 / 151 ) processed
                         ID               Name  Likes  Rating  Tips
0  4be74a502468c928505a0243  Taste Of India II     29     8.1    14
( 125 / 151 ) processed
                         ID            Name  Likes  Rating  Tips
0  5272ca4511d22488f6895caf  Drunken Munkey    208     8.6    61
( 126 / 151 ) processed
                         ID                    Name  Likes  Rating  Tips
0  56ed855a498ef3bb022352c3  Mughlai Indian Cuisine     26     7.8     9
( 127 / 151 ) processed
                         ID              Name  Likes  Rating  Tips
0  5b0c8e2d2f97ec002c67a428  Ashoka Fine Dine      9     7.8     5
( 128 / 151 ) processed
                         ID              Name  Likes  Rating  Tips
0  4bbb9dbded7776b0e1ad3e51  Tamarind TriBeCa    589     9.1   146
( 129 / 151 ) processed
                         ID                   Name  Likes  Rating  Tips
0  49d91c12f964a520015e1fe3  The Kati Roll Company    837     8.8   260
( 130 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4b65f2e3f964a5206e0a2be3
( 131 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4b65f2e3f964a5206e0a2be3
( 132 / 151 ) processed
                         ID          Name  Likes  Rating  Tips
0  4bad49a0f964a52041423be3  Annam Brahma     15     8.0     3
( 133 / 151 ) processed
                         ID              Name  Likes  Rating  Tips
0  4c434b2bd691c9b6ef8f8f0a  Sagar Restaurant     10     7.5     6
( 134 / 151 ) processed
                         ID            Name  Likes  Rating  Tips
0  4cc08b0900d83704ed474b5c  Sybil's Bakery      8     7.2     3
( 135 / 151 ) processed
                         ID         Name  Likes  Rating  Tips
0  551b7f75498e86c00a0ed2e1  Hungry Bird      8     6.9     3
( 136 / 151 ) processed
                         ID                           Name  Likes  Rating  \
0  4a63bfb4f964a520b3c51fe3  Chola Eclectic Indian Cuisine    153     8.3   

   Tips  
0    56  
( 137 / 151 ) processed
                         ID              Name  Likes  Rating  Tips
0  4fe4fb50c2eee335e4fea69d  Moti Mahal Delux    185     8.4    82
( 138 / 151 ) processed
                         ID                   Name  Likes  Rating  Tips
0  4f57f98fe4b0bd50f6bb8b31  The Kati Roll Company    182     8.2    55
( 139 / 151 ) processed
                         ID                   Name  Likes  Rating  Tips
0  59fcd48c464d6567ed2f5e37  The Kati Roll Company     18     8.6     2
( 140 / 151 ) processed
                         ID   Name  Likes  Rating  Tips
0  5ac3984ae57ca64be4dc6168  INDAY     33     8.1     9
( 141 / 151 ) processed
                         ID                   Name  Likes  Rating  Tips
0  4f57f98fe4b0bd50f6bb8b31  The Kati Roll Company    182     8.2    55
( 142 / 151 ) processed
                         ID                   Name  Likes  Rating  Tips
0  59fcd48c464d6567ed2f5e37  The Kati Roll Company     18     8.6     2
( 143 / 151 ) processed
                         ID   Name  Likes  Rating  Tips
0  5ac3984ae57ca64be4dc6168  INDAY     33     8.1     9
( 144 / 151 ) processed
                         ID            Name  Likes  Rating  Tips
0  4c48da9f3013a59356c5f0e1  Saffron Garden     17     7.5    16
( 145 / 151 ) processed
                         ID  Name  Likes  Rating  Tips
0  5b931ea69d7468002c3b1382  Adda    113     9.0    34
( 146 / 151 ) processed
                         ID             Name  Likes  Rating  Tips
0  5539753f498edbace4746b67  Tandoori Masala     12     7.7     2
( 147 / 151 ) processed
                         ID                Name  Likes  Rating  Tips
0  564d283d498e6e851df79d87  Great Indian Curry      3     6.8     2
( 148 / 151 ) processed
                         ID               Name  Likes  Rating  Tips
0  4b1b341bf964a5208af923e3  Five Star Banquet     30     7.1    31
( 149 / 151 ) processed
                         ID                  Name  Likes  Rating  Tips
0  50a287a7e4b0033f830f06db  Raj's Indian Kitchen     22     7.3     9
( 150 / 151 ) processed
Empty DataFrame
Columns: [ID, Name, Likes, Rating, Tips]
Index: []
No data available for id= 4b65f2e3f964a5206e0a2be3
( 151 / 151 ) processed